In [58]:
import pyspark
from pyspark.sql.functions import col

In [59]:
dataframe = sqlContext.read.csv('data/events.csv')

In [60]:
type(dataframe)

pyspark.sql.dataframe.DataFrame

In [61]:
# Versión poco eficiente en memoria ya que el sort se realiza fuera del cluster (creo).
# Esto lo hago para verificar el resultado posterior, para chequear lo que haga de forma Sparkstica.

dataframe.groupBy('_c1').count().sort(col('count').desc()).show()

+-----------------+------+
|              _c1| count|
+-----------------+------+
|   viewed product|528931|
|    brand listing| 98635|
|     visited site| 87378|
|  ad campaign hit| 82827|
|  generic listing| 67534|
|searched products| 56073|
|search engine hit| 50957|
|         checkout| 33735|
|       staticpage|  3598|
|       conversion|  1172|
|             lead|   448|
|            event|     1|
+-----------------+------+



In [62]:
# Vamos a tratar al dataframe como un RDD, y aplicar lo visto en clase de utilizar tuplas para poder contar
# y ordenar los resultados con reduceByKey y un takeOrdered.

dataframe.rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda a,b: a+b).takeOrdered(1, lambda x: -x[1])

[('viewed product', 528931)]

In [63]:
# Aplicamos la misma idea, sólo que antes filtramos por tipo de evento.
dataframe.rdd.filter(lambda x: x[1] == 'viewed product')\
    .map(lambda x: (x[4], 1)).reduceByKey(lambda a,b: a+b).takeOrdered(5, lambda x: -x[1])

[('2830.0', 4007),
 ('2829.0', 3466),
 ('2831.0', 3423),
 ('8443.0', 3265),
 ('12758.0', 2714)]